In [2]:
import os

# preprocess data

In [6]:
#--validate : if yo want rescaling 

!rm -r data/val
!rm -r data/train
!/opt/anaconda3/bin/python preprocess_dataset.py #--validate
!du -sh --human-readable data/*
!cat logs.txt

Validate: False
Number images ./data/train 418
Number images ./data/val 100
multi 36.484442472457886
32M	data/images
886M	data/train
16K	data/train.csv
182M	data/val
4.0K	data/val.csv
./data/train directory created.
Start preprocessing images
Writing target file ./data/train/1676_008.png
Writing target file ./data/train/5622_013.png
Writing target file ./data/train/0717_023.png
Writing target file ./data/train/5814_092.png
Writing target file ./data/train/1852_024.png
Writing target file ./data/train/1589_015.png
Writing target file ./data/train/2117_040.png
Writing target file ./data/train/6581_209.png
Writing target file ./data/train/9520_041.png
Writing target file ./data/train/9534_058.png
Writing target file ./data/train/0725_026.png
Writing target file ./data/train/5367_007.png
Writing target file ./data/train/9508_067.png
Writing target file ./data/train/0101_003.png
Writing target file ./data/train/0203_075.png
Writing target file ./data/train/5112_044.png
Writing target file .

# create tfdata

In [7]:
# modify csv with column names image_id,xmin,ymin,xmax,ymax,label
!lumi dataset transform \
      --type csv \
      --data-dir data/ \
      --output-dir ./tfdata/ \ # tfdata-scaled # i have 2 datasets  tfdata and tfdata-scaled
      --split train --split val \
      --only-classes=table
# split is the name you wish

INFO:tensorflow:Saving split "train" in output_dir = ./tfdata/
  [####################################]  100%             
INFO:tensorflow:Saved 338 records to "./tfdata/train.tfrecords"
INFO:tensorflow:Composition per class (train):
INFO:tensorflow:	table: 418
INFO:tensorflow:Saving split "val" in output_dir = ./tfdata/
  [####################################]  100%
INFO:tensorflow:Saved 65 records to "./tfdata/val.tfrecords"
INFO:tensorflow:Composition per class (val):
INFO:tensorflow:	table: 100


# Train the network 

In [ ]:
# I have 2 config files

!lumi train -c config-validate.yml 2> with_validate.log
!lumi train -c config.yml 2> without_validate.log

# Evaluate the networks

In [9]:
# evalute the no scaling network

# split is the name you wish
!lumi eval --split val -c config.yml

Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.from_tensor_slices(string_tensor).shuffle(tf.shape(input_tensor, out_type=tf.int64)[0]).repeat(num_epochs)`. If `shuffle=False`, omit the `.shuffle(...)`.
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.from_tensor_slices(input_tensor).shuffle(tf.shape(input_tensor, out_type=tf.int64)[0]).repeat(num_epochs)`. If `shuffle=False`, omit the `.shuffle(...)`.
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.from_tensors(tensor).repeat(num_epochs)`.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.TFRecordDataset`.
Instructions for updating:
Create a `

In [10]:
# split is the name you wish
!lumi eval --split val -c config-validate.yml

Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.from_tensor_slices(string_tensor).shuffle(tf.shape(input_tensor, out_type=tf.int64)[0]).repeat(num_epochs)`. If `shuffle=False`, omit the `.shuffle(...)`.
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.from_tensor_slices(input_tensor).shuffle(tf.shape(input_tensor, out_type=tf.int64)[0]).repeat(num_epochs)`. If `shuffle=False`, omit the `.shuffle(...)`.
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.from_tensors(tensor).repeat(num_epochs)`.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.TFRecordDataset`.
Instructions for updating:
Create a `

# Create checkpoints

In [11]:
!lumi checkpoint create \
    config.yml \
    -e name="witout validation" \
    -e alias=without

!lumi checkpoint create \
    config-validate.yml \
    -e name="with validation" \
    -e alias=with
!lumi checkpoint list
#18385f994797 without
#ef96ad036cce with

Creating checkpoint for given configuration...
Checkpoint 18385f994797 created successfully.
Creating checkpoint for given configuration...
Checkpoint ef96ad036cce created successfully.


# Load detector

In [7]:
# If no checkpoint specified, will assume `accurate` by default. In this case,
# we want to use our traffic checkpoint. The Detector can also take a config
# object.

from luminoth import Detector, read_image, vis_objects

#checkpoint='ef96ad036cce' # with rescaling
checkpoint='18385f994797' # without rescaling
detector = Detector(checkpoint=checkpoint, prob=.7)


INFO:tensorflow:Restoring parameters from /home/elsallab/.luminoth/checkpoints/18385f994797/model.ckpt-3380
INFO:tensorflow:Loaded checkpoint.


# Process and image
sample images are under data/visualization

In [22]:
from preprocess_dataset import preprocess_image
import cv2

#image_example = "0101_003 org.png"
#image_example = "2306_093.png"
image_example = "250091970_0.tiff"
image_example = "250091970_1.tiff"
image_example = "250091970_2.tiff"
image_example = "250091970_3.tiff"
image_example = "250091970_4.tiff"
image_example = "250091970_5.tiff"
image_example = "250091970_6.tiff"
image_example = "250091970_7.tiff"
image_example = "250091970_8.tiff"
image_example = "250091970_9.tiff"
image_example = "250091970_10.tiff"
image_example = "250091970_11.tiff"
image_example = "250091970_12.tiff"
image_example = "250091970_13.tiff"

processed = preprocess_image(cv2.imread("data/visualization/{}".format(image_example)),True)
cv2.imwrite("/tmp/valid-processed.png", processed)

processed = preprocess_image(cv2.imread("data/visualization/{}".format(image_example)),False)
cv2.imwrite("/tmp/processed.png", processed)

True

# Visualize

In [23]:
#image_file = '/tmp/valid-processed.png' # with rescaling
image_file =  '/tmp/processed.png' # without rescaling

image = read_image(image_file)

import time
start=time.time()
objects = detector.predict(image)
print(time.time()-start)
start=time.time()
objects = detector.predict(image)
print(time.time()-start)
print(objects)

output_image_name='data/visualization/{}'.format(image_example.split(".")[0]+"-tables."+image_example.split(".")[1])
print("Generating:",output_image_name)
vis_objects(read_image("data/visualization/{}".format(image_example)), objects).save(output_image_name)

import cv2
cv2.imwrite(output_image_name,cv2.imread(output_image_name))


#from IPython.display import display, Image
#i = Image(filename='data/visualization/{}'.format(output_image_name))
#display(i)

1.7684354782104492
0.9020862579345703
[{'bbox': [533, 4212, 5576, 4901], 'label': 'table', 'prob': 0.8498}, {'bbox': [659, 2043, 5347, 3704], 'label': 'table', 'prob': 0.7556}, {'bbox': [550, 560, 5425, 1848], 'label': 'table', 'prob': 0.7169}]
Generating: data/visualization/250091970_7-tables.tiff


True

In [62]:
!nvidia-smi


Tue Dec  4 04:11:47 2018       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 396.51                 Driver Version: 396.51                    |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  GeForce GTX 108...  Off  | 00000000:17:00.0 Off |                  N/A |
| 20%   32C    P8    17W / 250W |     12MiB / 11178MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
|   1  GeForce GTX 108...  Off  | 00000000:65:00.0  On |                  N/A |
| 20%   40C    P8    18W / 250W |    955MiB / 11177MiB |      4%      Default |
+-------------------------------+----------------------+----------------------+
        

In [6]:
!/opt/anaconda3/bin/pip install tensorflow-gpu --user --upgrade

from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

Tue Dec  4 02:48:59 2018       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 396.51                 Driver Version: 396.51                    |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  GeForce GTX 108...  Off  | 00000000:17:00.0 Off |                  N/A |
| 20%   35C    P8    17W / 250W |  10844MiB / 11178MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
|   1  GeForce GTX 108...  Off  | 00000000:65:00.0  On |                  N/A |
| 20%   42C    P8    18W / 250W |  10889MiB / 11177MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
        

In [ ]:

uploaded = GoogleDrive(gauth).CreateFile({'title': title, "parents": [{"kind": "drive#fileLink", "id": self.project_id["owned by me"]}]})
uploaded.SetContentFile(file_path)  # file on disk
uploaded.Upload()